In [1]:
import simpy
from numpy.random import choice

In [2]:
n=5 #Prime router number
N=3 #regular router number
m=3 #shortest path number

In [3]:
class Packet(object):
    """ A very simple class that represents a packet.
        This packet will run through a queue at a switch output port.
        We use a float to represent the size of the packet in bytes so that
        we can compare to ideal M/M/1 queues.

        Parameters
        ----------
        time : float
            the time the packet arrives at the output queue.
        size : float
            the size of the packet in bytes
        id : int
            an identifier for the packet
        src, dst : int
            identifiers for source and destination

    """
    def __init__(self, packet_id, path, cur_path_ind, time, size, src, dst):
        
        self.packet_id=packet_id
        self.path=path
        self.cur_path_ind=cur_path_ind
        
        
        self.time = time
        self.size = size
        self.src = src
        self.dst = dst


    def __repr__(self):
        return "id: {}, src: {}, time: {}, size: {}".\
            format(self.packet_id, self.src, self.time, self.size)

'''path, cur_path_ind, '''

In [4]:
class PacketGenerator(object):
    """ Generates packets with given inter-arrival time distribution.
        Set the "out" member variable to the entity to receive the packet.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        adist : function
            a no parameter function that returns the successive inter-arrival times of the packets
        sdist : function
            a no parameter function that returns the successive sizes of the packets
        initial_delay : number
            Starts generation after an initial delay. Default = 0
        finish : number
            Stops generation at the finish time. Default is infinite


    """
    def __init__(self, env, gen_prob, generator_id,  adist, sdist, initial_delay=0, finish=float("inf")):
        
        self.gen_prob=gen_prob
        self.generator_id = generator_id
        self.env = env
        self.adist = adist
        self.sdist = sdist
        self.initial_delay = initial_delay
        self.finish = finish
        self.out = None
        self.packets_sent = 0
        self.action = env.process(self.run())  # starts the run() method as a SimPy process


    def run(self):
        """The generator function used in simulations.
        """
        
        yield self.env.timeout(self.initial_delay)
        while self.env.now < self.finish:
            # wait for next transmission
            yield self.env.timeout(self.adist())
            
            destination = choice(range(1,N+1), p=self.gen_prob)
            print('destination',destination)
            #print('id',self.generator_id)
            opt_path=path[self.generator_id][destination]
            
            print('opt_path',opt_path)
            
            self.packets_sent += 1
            
            
            pkt = Packet(packet_id=self.packets_sent, 
                         path=opt_path, 
                         cur_path_ind=0, 
                         time=self.env.now, 
                         size=self.sdist(), 
                         src=self.generator_id, 
                         dst=destination)
            
            #if next_node==destination:
            #    sink[destination].put(p)
            #else:
            #    switch[destination].put(p)
            
            self.out.put(pkt)
       

In [5]:
class PacketSink(object):
    """ Receives packets and collects delay information into the
        waits list. You can then use this list to look at delay statistics.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        debug : boolean
            if true then the contents of each packet will be printed as it is received.
        rec_arrivals : boolean
            if true then arrivals will be recorded
        absolute_arrivals : boolean
            if true absolute arrival times will be recorded, otherwise the time between consecutive arrivals
            is recorded.
        rec_waits : boolean
            if true waiting time experienced by each packet is recorded
        selector: a function that takes a packet and returns a boolean
            used for selective statistics. Default none.

    """
    def __init__(self, env, rec_arrivals=False, absolute_arrivals=False, rec_waits=True, debug=False, selector=None):
        self.store = simpy.Store(env)
        self.env = env
        self.rec_waits = rec_waits
        self.rec_arrivals = rec_arrivals
        self.absolute_arrivals = absolute_arrivals
        self.waits = []
        self.arrivals = []
        self.debug = debug
        self.packets_rec = 0
        self.bytes_rec = 0
        self.selector = selector
        self.last_arrival = 0.0

    def put(self, pkt):
        #print('packet_id',pkt.id)
        print('sink',pkt)
        
        if not self.selector or self.selector(pkt):
            now = self.env.now
            if self.rec_waits:
                self.waits.append(self.env.now - pkt.time)
            if self.rec_arrivals:
                if self.absolute_arrivals:
                    self.arrivals.append(now)
                else:
                    self.arrivals.append(now - self.last_arrival)
                self.last_arrival = now
            self.packets_rec += 1
            self.bytes_rec += pkt.size
            if self.debug:
                print(pkt)

In [6]:
class SwitchPort(object):
    """ Models a switch output port with a given rate and buffer size limit in bytes.
        Set the "out" member variable to the entity to receive the packet.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        rate : float
            the bit rate of the port
        qlimit : integer (or None)
            a buffer size limit in bytes or packets for the queue (including items
            in service).
        limit_bytes : If true, the queue limit will be based on bytes if false the
            queue limit will be based on packets.

    """
    def __init__(self, env, router_id, rate, qlimit=None, limit_bytes=True, debug=False):
        self.id=router_id
        self.store = simpy.Store(env)
        self.rate = rate
        self.env = env
        self.out = None
        self.packets_rec = 0
        self.packets_drop = 0
        self.qlimit = qlimit
        self.limit_bytes = limit_bytes
        self.byte_size = 0  # Current size of the queue in bytes
        self.debug = debug
        self.busy = 0  # Used to track if a packet is currently being sent
        self.action = env.process(self.run())  # starts the run() method as a SimPy process

    def run(self):
        while True:
            msg = (yield self.store.get())
            print('self.store',self.store)
            self.busy = 1
            self.byte_size -= msg.size
            yield self.env.timeout(msg.size*8/self.rate)#(msg.size/self.rate)
            #self.out.put(msg)
            
                
            path=msg.path
            cur_path_ind=msg.cur_path_ind
            destination=msg.dst
                
            #print('path',path)
            #print('cur_path_ind',cur_path_ind)
            #print('destination',destination)
            #print('path[cur_path_ind]',path[cur_path_ind])
            
            #self.packet_id=packet_id
            #self.path=path
            #self.cur_path_ind=cur_path_ind
            print('msg packet_id src ',msg.packet_id, msg.src)
                
            if path[cur_path_ind]==destination:
                print('send to sink' + str(destination))
                sink[destination].put(msg)
            else:
                next_node=path[cur_path_ind+1]
                print('send to switch' + str(next_node))
                msg.cur_path_ind=cur_path_ind+1
                #print('msg.cur_path_ind_after',msg.cur_path_ind)
                
                '''
                pck = Packet(packet_id=msg.packet_id, 
                         path=msg.path, 
                         cur_path_ind=msg.cur_path_ind+1, 
                         time=msg.time, 
                         size=msg.size, 
                         src=msg.src, 
                         dst=msg.dst)
                
                print('pck.cur_path_ind',pck.cur_path_ind)
                '''
                
                print('msg', msg.packet_id, msg.src)
                switch[next_node].put(msg)
                
            self.busy = 0
            if self.debug:
                print(msg)

    def put(self, pkt):
        self.packets_rec += 1
        tmp_byte_count = self.byte_size + pkt.size

        if self.qlimit is None:
            self.byte_size = tmp_byte_count
            return self.store.put(pkt)
        if self.limit_bytes and tmp_byte_count >= self.qlimit:
            self.packets_drop += 1
            return
        elif not self.limit_bytes and len(self.store.items) >= self.qlimit-1:
            self.packets_drop += 1
        else:
            self.byte_size = tmp_byte_count
            return self.store.put(pkt)
            
                

In [7]:
from random import expovariate
import simpy

def constArrival():  # Constant arrival distribution for generator 1
    return 1.5

def constArrival2():
    return 2.0

def distSize():
    return expovariate(0.01)

In [8]:
def constArrival():
    return 1.5    # time interval

def constSize():
    return 100.0  # bytes

In [9]:
path=[0]*4
prob=[0]*4
for i in range(4):
    path[i]=[0]*4


path[1][2]=[1,3,2]
path[2][3]=[2,1,3]
path[3][1]=[3,2,1]
path[2][1]=[2,3,1]
path[3][2]=[3,1,2]
path[1][3]=[1,2,3]

gen_prob=[0]*4
gen_prob[1]=[0,0.5,0.5]
gen_prob[2]=[0.2,0,0.8]
gen_prob[3]=[0.7,0.3,0]

In [10]:
path

[[0, 0, 0, 0],
 [0, 0, [1, 3, 2], [1, 2, 3]],
 [0, [2, 3, 1], 0, [2, 1, 3]],
 [0, [3, 2, 1], [3, 1, 2], 0]]

In [11]:
generators=[0]*4
sink=[0]*4
switch=[0]*4

env = simpy.Environment()  # Create the SimPy environment

for i in range(1,4):
    generators[i]=PacketGenerator(env, gen_prob[i], i, constArrival, constSize)
    switch[i]=SwitchPort(env, i, rate=200.0, qlimit=300, debug=True)
    generators[i].out = switch[i]
    
    sink[i]=PacketSink(env, debug=True)

    
env.run(until=2000)

for i in range(1,4):
    print("sink {} waits: {}".format(i, sink[i].waits))
    print("{} received: {}, dropped {}, sent {}".format(i, sink[i].packets_rec,
         switch[i].packets_drop, generators[i].packets_sent))

destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  1 1
send to switch3
msg 1 1
id: 1, src: 1, time: 1.5, size: 100.0
msg packet_id src  1 2
send to switch1
msg 1 2
id: 1, src: 2, time: 1.5, size: 100.0
msg packet_id src  1 3
send to switch2
msg 1 3
id: 1, src: 3, time: 1.5, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 2
opt_path [1, 3

self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 2
opt_path [3, 1, 2]
msg packet_id src  122 1
send to switch2
msg 122 1
id: 122, src: 1, time: 183.0, size: 100.0
msg packet_id src  122 2
send to switch1
msg 122 2
id: 122, src: 2, time: 183.0, size: 100.0
msg packet_id src  122 3
send to switch2
msg 122 3
id: 122, src: 3, time: 183.0, size: 100.0
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destinat

opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  236 1
send to switch3
msg 236 1
id: 236, src: 1, time: 354.0, size: 100.0
msg packet_id src  236 2
send to switch3
msg 236 2
id: 236, src: 2, time: 354.0, size: 100.0
msg packet_id src  236 3
send to switch2
msg 236 3
id: 236, src: 3, time: 354.0, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  239 1
send to switch3
msg 239 1
id: 239, src

msg 356 2
id: 356, src: 2, time: 534.0, size: 100.0
msg packet_id src  356 3
send to switch2
msg 356 3
id: 356, src: 3, time: 534.0, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  359 1
send to switch2
msg 359 1
id: 359, src: 1, time: 538.5, size: 100.0
msg packet_id src  359 2
send to switch1
msg 359 2
id: 359, src: 2, time: 538.5, size: 100.0
msg packet_id src  359 3
send to switch2
msg 359 3
id: 359, src: 3, time: 538.5, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>

destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  487 1
send to switch2
msg 487 1
id: 487, src: 1, time: 730.5, size: 100.0
msg packet_id src  487 2
send to switch1
msg 487 2
id: 487, src: 2, time: 730.5, size: 100.0
msg packet_id src  487 3
send to switch2
msg 487 3
id: 487, src: 3, time: 730.5, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  490 1
send to switch3
msg 490 

...

msg 722 3
id: 722, src: 3, time: 1083.0, size: 100.0
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 2
opt_path [3, 1, 2]
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  724 1
send to switch2
msg 724 1
id: 724, src: 1, time: 1086.0, size: 100.0
msg packet_id src  724 2
send to switch1
msg 724 2
id: 724, src: 2, time: 1086.0, size: 100.0
msg packet_id src  724 3
send to switch2
msg 724 3
id: 724, src: 3, time: 1086.0, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.

destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  839 1
send to switch2
msg 839 1
id: 839, src: 1, time: 1258.5, size: 100.0
msg packet_id src  839 2
send to switch1
msg 839 2
id: 839, src: 2, time: 1258.5, size: 100.0
msg packet_id src  839 3
send to switch2
msg 839 3
id: 839, src: 3, time: 1258.5, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src  842 1
send to switch3
msg 842 1
id: 842, src: 1, time: 1263.0, size: 100.0
msg packet_id src  842 2
send to switch3
msg 842 2
id: 842, src: 2, time: 1263.0, size: 100.0
msg packet_id src  842 3
send to switch1
msg 842 3
id: 8

opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 2
opt_path [1, 3, 2]
destination 1
opt_path [2, 3, 1]
destination 1
opt_path [3, 2, 1]
msg packet_id src  959 1
send to switch3
msg 959 1
id: 959, src: 1, time: 1438.5, size: 100.0
msg packet_id src  959 2
send to switch1
msg 959 2
id: 959, src: 2, time: 1438.5, size: 100.0
msg packet_id src  959 3
send to switch2
msg 959 3
id: 959, src: 3, time: 1438.5, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 2
opt_path [3, 1, 2]
msg packet_id src  962 1
send to switch3
msg 962 1
id: 962, src: 1, time: 1443.0, size: 100.0
msg packet_id src  962 2
send to switch1
msg 962 2
id: 962, src: 

destination 1
opt_path [3, 2, 1]
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 2
opt_path [3, 1, 2]
msg packet_id src  1079 1
send to switch3
msg 1079 1
id: 1079, src: 1, time: 1618.5, size: 100.0
msg packet_id src  1079 2
send to switch3
msg 1079 2
id: 1079, src: 2, time: 1618.5, size: 100.0
msg packet_id src  1079 3
send to switch2
msg 1079 3
id: 1079, src: 3, time: 1618.5, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 2
opt_path [3, 1, 2]
destination 3
opt_path [1, 2, 3]
destination 1
opt_path [2, 3, 1]
destination 2
opt_path [3, 1, 2]
msg packet_id src  1082 1
send to switch3
msg 1082 1
id: 1082, src: 1, time: 1623.0, size: 100.0
msg packet_id src  1082 2
send to switch1
msg 1082 2
id: 1082, src: 2, t

opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 2
opt_path [3, 1, 2]
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 2
opt_path [3, 1, 2]
msg packet_id src  1196 1
send to switch3
msg 1196 1
id: 1196, src: 1, time: 1794.0, size: 100.0
msg packet_id src  1196 2
send to switch1
msg 1196 2
id: 1196, src: 2, time: 1794.0, size: 100.0
msg packet_id src  1196 3
send to switch2
msg 1196 3
id: 1196, src: 3, time: 1794.0, size: 100.0
self.store <simpy.resources.store.Store object at 0x0000023637E19408>
self.store <simpy.resources.store.Store object at 0x0000023637F751C8>
self.store <simpy.resources.store.Store object at 0x0000023637F75F88>
destination 3
opt_path [1, 2, 3]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
destination 2
opt_path [1, 3, 2]
destination 3
opt_path [2, 1, 3]
destination 1
opt_path [3, 2, 1]
msg packet_id src 

In [12]:
env = simpy.Environment()  # Create the SimPy environment
ps = PacketSink(env, debug=True) # debug: every packet arrival is printed
pg = PacketGenerator(env, "SJSU", constArrival, constSize)
switch_port = SwitchPort(env, rate=200.0, qlimit=300)
switch_port2 = SwitchPort(env, rate=200.0, qlimit=300)
switch_port3 = SwitchPort(env, rate=200.0, qlimit=300)
# Wire packet generators and sinks together
pg.out = switch_port
switch_port.out = switch_port2
switch_port2.out = switch_port3
switch_port3.out = ps
env.run(until=20)
print("waits: {}".format(ps.waits))
print("received: {}, dropped {}, sent {}".format(ps.packets_rec,
     switch_port.packets_drop, pg.packets_sent))


TypeError: __init__() missing 1 required positional argument: 'sdist'

In [ ]:
from numpy.random import choice

In [ ]:
prob=[0.5,0.5,0]

In [ ]:
draw = choice([3,4,5], 
              p=prob)

In [ ]:
draw